# Get SNF data

The data was downloaded manually directly from the National Science Foundation awards site in XML format by fiscal year, since downloads are limited to 3000 records.

In [1]:
import pandas as pd
import io
import glob
import xml.etree.ElementTree as ET

pd.set_option('display.max_columns',500)
pd.set_option('display.max_rows', 500)
# set wider prints for pd
pd.options.display.max_colwidth = 250

# set filterwarnings to 'once' to show warnings only the first time code is run
import warnings
warnings.filterwarnings('ignore') # show warnings only the first time code is run

# pretty print all cell's output and not just the last one
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
# Write the pattern: pattern
pattern = '*.xml'

xml_files = glob.glob('./data/2016/'+pattern)

# Print the file names
len(xml_files)
print(xml_files[:1])

tree = ET.parse('./data/2016/1607387.xml')
root = tree.getroot()
snf = [elem.text for elem in root.iter()]
column_names= [elem.tag for elem in root.iter()]
column_names
df_snf = pd.DataFrame(snf).T
df_snf.columns = column_names
df_snf.head(1)

12595

['./data/2016/1607387.xml']


['rootTag',
 'Award',
 'AwardTitle',
 'AwardEffectiveDate',
 'AwardExpirationDate',
 'AwardTotalIntnAmount',
 'AwardAmount',
 'AwardInstrument',
 'Value',
 'Organization',
 'Code',
 'Directorate',
 'Abbreviation',
 'LongName',
 'Division',
 'Abbreviation',
 'LongName',
 'ProgramOfficer',
 'SignBlockName',
 'AbstractNarration',
 'MinAmdLetterDate',
 'MaxAmdLetterDate',
 'ARRAAmount',
 'AwardID']

,rootTag,Award,AwardTitle,AwardEffectiveDate,AwardExpirationDate,AwardTotalIntnAmount,AwardAmount,AwardInstrument,Value,Organization,Code,Directorate,Abbreviation,LongName,Division,Abbreviation,LongName,ProgramOfficer,SignBlockName,AbstractNarration,MinAmdLetterDate,MaxAmdLetterDate,ARRAAmount,AwardID
0,\n,\n,The NuPRISM Detector: Extending the Sensitivity of Neutrino Oscillation Experiments,09/01/2016,08/31/2019,240000.00,240000,\n,Continuing grant,\n,03010000,\n,MPS,Direct For Mathematical & Physical Scien,\n,PHY,Division Of Physics,\n,Randy Ruchti,"This award supports work to understand neutrinos: the smallest but most common massive particle in the universe. Although there are a billion neutrinos for every more well-known electron or proton, neutrinos interact only via the so-called ""weak""...",07/05/2016,07/13/2018,None,1607387


In [3]:
def get_files(files_path, pattern = '*.xml'):
    # Write the pattern: pattern
    #pattern = '*.xml'
    # get all the xml files into a list
    xml_files = glob.glob(files_path+pattern)
    # return list of files
    return xml_files

In [4]:
def get_valid_columns(xml_files_list, extract_columns_from_file=5):
    tree = ET.parse(xml_files[exract_columns_from_file])
    root = tree.getroot()
    l = [elem.text for elem in root.iter()]
    df = pd.DataFrame(l).T
    columns = [elem.tag for elem in root.iter()]
    return columns

In [5]:
# choose only valid columns
valid_columns = ['AwardID','AwardTitle', 'AwardEffectiveDate','AwardExpirationDate', 'AwardTotalIntnAmount', 'SignBlockName','AwardAmount', 'LongName',
                     'Value','AbstractNarration','Institution','Name', 'CityName', 'ZipCode','CountryName', 'StateName', 'StateCode']


def read_xml(list_xml_files, valid_columns):
    final = []
    df = pd.DataFrame().T
    for i in list_xml_files: #for i in xml_files[:1]:
        path = "'"+str(i)+"'"
        #print('reading ' + path)
        try:
            tree = ET.parse(i)
            root = tree.getroot()
            snf = [elem.text for elem in root.iter()]
            column_names= [elem.tag for elem in root.iter()]
            df_snf = pd.DataFrame(snf).T
            df_snf.columns = column_names
            df_snf = df_snf.loc[:,valid_columns]
        #df_snf.info()
            df_snf = df_snf.astype(str) # convert all data to same type to make concatanation easier
            df= pd.concat([df,df_snf], axis=0, sort = False)  
        except:
            pass
        #final.append(df_snf)
    return df

In [6]:
save_path = "./data/snf_"

In [7]:
xml_files_2015 = get_files('./data/2015/', '*.xml')

In [8]:
len(xml_files_2015)

13074

In [9]:
snf_2015 = read_xml(xml_files_2015, valid_columns)

In [10]:
snf_2015.head()

,AwardID,AwardTitle,AwardEffectiveDate,AwardExpirationDate,AwardTotalIntnAmount,SignBlockName,AwardAmount,LongName,LongName,Value,AbstractNarration,Institution,Name,CityName,ZipCode,CountryName,StateName,StateCode
0,1522829,"Exploring the Linkages among Mid-Latitude Dynamics, Clouds, and Climate Sensitivity",08/01/2015,01/31/2020,298314.00,Ming Cai,298314,Directorate For Geosciences,Div Atmospheric & Geospace Sciences,Standard Grant,The goal of this research is to better understand the connection between mid-latitude clouds and dynamics in observations. The reasons for why the such linkages are well-represented in some global climate models but not in others will be studied....,\n,University of Virginia Main Campus,CHARLOTTESVILLE,229044195,United States,Virginia,VA
0,1555479,Studies of Optical and Chemical Properties of Biomass Burning Aerosols,02/15/2016,09/30/2019,488281.00,Sylvia Edgerton,488281,Directorate For Geosciences,Div Atmospheric & Geospace Sciences,Standard Grant,This project involves laboratory studies to assess the chemical composition and optical properties of small atmospheric particles created from biomass burning. The investigators are focusing on common East African biomass sources including eucaly...,\n,North Carolina Agricultural & Technical State University,Greensboro,274110001,United States,North Carolina,NC
0,1540712,UR Noyce Master Teaching Fellows Program--Phase II: Leveraging Unique Opportunities to Develop STEM Teacher Leaders for Urban Schools,09/01/2015,08/31/2020,nan,Kathleen B. Bergin,2046998,Direct For Education and Human Resources,Division Of Undergraduate Education,Standard Grant,"This Noyce Master Teaching Fellowship project will prepare a cadre of 14, K-12 STEM master teachers for high-need urban districts in upstate New York. Along with standard professional development activities, Master Teaching Fellows (MTFs) will ...",\n,University of Rochester,Rochester,146270140,United States,New York,NY
0,1519667,IBSS-Ex: Exploring Recidivism Through a Tablet-Based Battery to Assess Individual Decision Making,08/15/2015,01/31/2017,nan,Thomas J. Baerwald,244639,"Direct For Social, Behav & Economic Scie",SBE Off Of Multidisciplinary Activities,Standard Grant,"Scientists long have explored and measured a range of decision-making traits, such as the ability to inhibit impulses or to understand other people's emotions. Researchers have developed, validated, and refined a wide range of neurocognitive ass...",\n,Baylor College of Medicine,HOUSTON,770303411,United States,Texas,TX
0,1550501,Alvins New Users Workshop,07/15/2015,12/31/2017,nan,Brian Midson,52953,Directorate For Geosciences,Division Of Ocean Sciences,Standard Grant,"This award provides funds for an early career development exercise using the Human Occupied Vehicle Alvin. The training will include a series of half-day, mini-courses on various aspects of Alvin dive operations, cruise planning and execution, a...",\n,Woods Hole Oceanographic Institution,WOODS HOLE,025431041,United States,Massachusetts,MA


In [11]:
snf_2015.shape
snf_2015.to_csv('./data/snf_2015.csv', index = False)

(13070, 18)

### Read 2017

In [12]:
xml_files_2017 = get_files('./data/2017/', '*.xml')
len(xml_files_2017)

12269

In [13]:
snf_2017 = read_xml(xml_files_2017, valid_columns)
snf_2017.shape

(12268, 18)

In [14]:
snf_2017.to_csv(save_path+'2017.csv', index = False)

### Read 2014

In [15]:
xml_files_2014 = get_files('./data/2014/', '*.xml')
len(xml_files_2014)

12109

In [16]:
snf_2014 = read_xml(xml_files_2014, valid_columns)
snf_2014.shape

(12109, 18)

In [17]:
snf_2014.to_csv(save_path+'2014.csv', index = False)

### Read 2013

In [18]:
xml_files_2013 = get_files('./data/2013/', '*.xml')
len(xml_files_2013)
snf_2013 = read_xml(xml_files_2013, valid_columns)

11503

In [19]:
snf_2013.shape
snf_2013.to_csv(save_path+'2013.csv', index = False)

(11502, 18)

### Read 2018

In [20]:
xml_files_2018 = get_files('./data/2018/', '*.xml')
len(xml_files_2018)
snf_2018 = read_xml(xml_files_2018, valid_columns)

11566

In [21]:
snf_2018 = read_xml(xml_files_2018, valid_columns)

In [22]:
snf_2018.shape
snf_2018.to_csv(save_path+'2018.csv', index = False)

(11564, 18)

### Read 2012

In [23]:
xml_files_2012 = get_files('./data/2012/', '*.xml')
len(xml_files_2012)
snf_2012 = read_xml(xml_files_2012, valid_columns)

12169

In [24]:
snf_2012.shape
snf_2012.to_csv(save_path+'2012.csv', index = False)

(12167, 18)

### Read 2011

In [25]:
xml_files_2011 = get_files('./data/2011/', '*.xml')
len(xml_files_2011)
snf_2011 = read_xml(xml_files_2011, valid_columns)

11652

In [26]:
snf_2011.shape
snf_2011.to_csv(save_path+'2011.csv', index = False)

(11649, 18)

### Read 2010

In [27]:
xml_files_2010 = get_files('./data/2010/', '*.xml')
len(xml_files_2010)
snf_2010 = read_xml(xml_files_2010, valid_columns)

13094

In [28]:
snf_2010.shape
snf_2010.to_csv(save_path+'2010.csv', index = False)

(12458, 18)

### Read 2009

In [29]:
xml_files_2009 = get_files('./data/2009/', '*.xml')
len(xml_files_2009)
snf_2009 = read_xml(xml_files_2009, valid_columns)

15433

In [30]:
snf_2009.shape
snf_2009.to_csv(save_path+'2009.csv', index = False)

(6909, 18)

### Read 2008

In [31]:
xml_files_2008 = get_files('./data/2008/', '*.xml')
len(xml_files_2008)
snf_2008 = read_xml(xml_files_2008, valid_columns)

12614

In [32]:
snf_2008.shape
snf_2008.to_csv(save_path+'2008.csv', index = False)

(7877, 19)

### Read 2007

In [33]:
xml_files_2007 = get_files('./data/2007/', '*.xml')
len(xml_files_2007)
snf_2007 = read_xml(xml_files_2007, valid_columns)

12221

In [34]:
snf_2007.shape
snf_2007.to_csv(save_path+'2007.csv', index = False)

(8270, 19)

### Read 2006

In [35]:
xml_files_2006 = get_files('./data/2006/', '*.xml')
len(xml_files_2006)
snf_2006 = read_xml(xml_files_2006, valid_columns)

11064

In [36]:
snf_2007.shape
snf_2007.to_csv(save_path+'2006.csv', index = False)

(8270, 19)

### Read 2005

In [37]:
xml_files_2005 = get_files('./data/2005/', '*.xml')
len(xml_files_2005)
snf_2005 = read_xml(xml_files_2005, valid_columns)

10726

In [38]:
snf_2005.shape
snf_2005.to_csv(save_path+'2005.csv', index = False)

(7388, 19)

### Read 2004

In [39]:
xml_files_2004 = get_files('./data/2004/', '*.xml')
len(xml_files_2004)
snf_2004 = read_xml(xml_files_2004, valid_columns)

10722

In [40]:
snf_2004.shape
snf_2004.to_csv(save_path+'2004.csv', index = False)

(7259, 19)

### Read 2003

In [41]:
xml_files_2003 = get_files('./data/2003/', '*.xml')
len(xml_files_2003)
snf_2003 = read_xml(xml_files_2003, valid_columns)

11557

In [42]:
snf_2003.shape
snf_2003.to_csv(save_path+'2003.csv', index = False)

(3128, 18)

### Read 2002

In [43]:
xml_files_2002 = get_files('./data/2002/')
len(xml_files_2002)
snf_2002 = read_xml(xml_files_2002, valid_columns)

10950

In [44]:
snf_2002.shape
snf_2002.to_csv(save_path+'2002.csv', index = False)

(7658, 19)

### Read 2001

In [45]:
xml_files_2001 = get_files('./data/2001/')
len(xml_files_2001)
snf_2001 = read_xml(xml_files_2001, valid_columns)

9847

In [46]:
snf_2001.shape
snf_2001.to_csv(save_path+'2001.csv', index = False)

(2230, 18)

### Read 2000

In [47]:
xml_files_2000 = get_files('./data/2000/')
len(xml_files_2000)
snf_2000 = read_xml(xml_files_2000, valid_columns)

10369

In [48]:
snf_2000.shape
snf_2000.to_csv(save_path+'2000.csv', index = False)

(7442, 19)

### Read 2016

In [49]:
xml_files_2016 = get_files('./data/2016/')
len(xml_files_2016)
snf_2016 = read_xml(xml_files_2016, valid_columns)

12595

In [50]:
snf_2016.shape
snf_2016.to_csv(save_path+'2016.csv', index = False)

(12595, 18)

## Merge all the data

In [57]:
data = get_files('./data/', '*.csv')
len(data)

19

In [58]:
# use a list comprehension to iterate through csv files, read them and append them to an empty list
snf_data_list = []
snf_append = [snf_data_list.append(pd.read_csv(file)) for file in data]
snf_data = pd.concat(snf_data_list)

In [59]:
snf_data.shape

(175813, 19)

In [60]:
snf_data.drop_duplicates(keep = 'first',inplace=True)

In [61]:
snf_data.shape

(167542, 19)

In [62]:
#snf_data.to_csv('./data/snf_data.csv', index = False)
snf_data.to_csv('../data/raw/snf_data.csv', index = False)
